In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd

from pathlib import Path

data_path = Path('/kaggle/input/h-and-m-personalized-fashion-recommendations/')

In [ ]:
transactions = pd.read_csv(
    data_path / 'transactions_train.csv',
    # set dtype or pandas will drop the leading '0' and convert to int
    dtype={'article_id': str} 
)
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])
transactions['month'] = transactions['t_dat'].dt.month

submission = pd.read_csv(data_path / 'sample_submission.csv')

In [ ]:
print(transactions.shape)
transactions.head()

In [ ]:
transactions['t_dat'].min(), transactions['t_dat'].max()

In [ ]:
transactions['month'].value_counts()

In [ ]:
# Filtering on date..inspired from this notebook https://www.kaggle.com/hengzheng/time-is-our-best-friend
# Only picking summer months as test period is 1 week after 22nd september
# transactions = transactions.loc[transactions['t_dat']>=datetime(2020, 9, 7)]
transactions = transactions.loc[transactions['month'].isin([4,5,6,7,8,9,10])]
transactions.reset_index(drop=True, inplace=True)
print(transactions.shape)

In [ ]:
purchase_dict = {}

for i,x in enumerate(zip(transactions['customer_id'], transactions['article_id'])):
    cust_id, art_id = x
    if cust_id not in purchase_dict:
        purchase_dict[cust_id] = {}
    
    if art_id not in purchase_dict[cust_id]:
        purchase_dict[cust_id][art_id] = 0
    
    purchase_dict[cust_id][art_id] += 1
    
print(len(purchase_dict))

In [ ]:
print(submission.shape)
submission.head()

In [ ]:
not_so_fancy_but_fast_benchmark = submission[['customer_id']]
prediction_list = []
dummy_list = list((transactions['article_id'].value_counts()).index)[:12]
dummy_pred = ' '.join(dummy_list)

for i, cust_id in enumerate(submission['customer_id'].values.reshape((-1,))):
    if cust_id in purchase_dict:
        l = sorted((purchase_dict[cust_id]).items(), key=lambda x: x[1], reverse=True)
        l = [y[0] for y in l]
        if len(l)>12:
            s = ' '.join(l[:12])
        else:
            s = ' '.join(l+dummy_list[:(12-len(l))])
    else:
        s = dummy_pred
    prediction_list.append(s)

not_so_fancy_but_fast_benchmark['prediction'] = prediction_list
print(not_so_fancy_but_fast_benchmark.shape)
not_so_fancy_but_fast_benchmark.head()

In [ ]:
not_so_fancy_but_fast_benchmark.to_csv('not_so_fancy_but_fast_benchmark.csv', index=False)